In [ ]:
import pandas as pd
import os

In [ ]:
combined_dir = "C:/Users/stell/OneDrive/desktop/CANS_LAB/compare_timings"
output_folder = "C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER"

# Define word columns and window size
word_columns = ['text', 'Word', 'Wav2vec2_word', 'MFA_word', 'Vosk_word']  # Replace with yours

timing_map = {
    'text': ['tmin', 'tmax'],
    'Word': ['Start', 'End'],
    'Wav2vec2_word': ['Wav2vec2_start', 'Wav2vec2_end'],
    'MFA_word': ['MFA_Start', 'MFA_End'],
    'Vosk_word': ['Vosk_Start', 'Vosk_End']
}

reference_col = 'text'  # Original timing
window_size = 5

In [5]:
for filename in os.listdir(combined_dir):
    
    if filename.lower().endswith('_combined.csv'):
    
        #make into a single path
        combined_path = os.path.join(combined_dir, filename)

        df_combined = pd.read_csv(combined_path)


        #apply function to each
        fixed = fix_misalignments(df_combined)

        base_name = os.path.splitext(filename)[0] + "_filtered"
        
        csv_path = os.path.join(output_folder, f"{base_name}.csv")

        #-----One time thing
        fixed.rename(columns = {
            'Start': 'WhisperX_start',
            'End': 'WhisperX_end'
        }, inplace = True)

        
        fixed.to_csv(csv_path, index=False, float_format="%.7f")
        #--------
        
        print(f"Saved CSV: {csv_path}")
        

C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Tem

Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F03_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F05_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F07_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F11_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F12_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F15_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F16_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F19_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F21_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LAB/BETTER\F22_single105_combined_filtered.csv
Saved CSV: C:/Users/stell/OneDrive/desktop/CANS_LA

C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Temp\ipykernel_9128\947887744.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
C:\Users\stell\AppData\Local\Tem

In [3]:
def fix_misalignments(df):
    
    
    # Lowercased version of words for matching
    df_words = df[word_columns].applymap(lambda x: str(x).lower() if isinstance(x, str) else '')
    
    # Prepare output rows
    aligned_rows = []
    word_counters = {}
    
    for i, ref_word in df_words[reference_col].items():
        if not ref_word:
            continue
    
        original_word = df.at[i, reference_col]  # preserve original casing

        word_key = f"{original_word}_{i}"

        match_info = {'Word': original_word}
        
        all_found = True
        for col in word_columns:
            found = False
            word_window = df_words[col].iloc[max(0, i - window_size): min(len(df), i + window_size + 1)]
            
            for j in word_window.index:
                
                if df_words.at[j, col] == ref_word:
                    col_word_key = f"{df.at[j, col]}_{j}"
                    if col_word_key not in word_counters:
                        # Store the timing info for this matched row
                        for timing_col in timing_map[col]:
                            match_info[f"{timing_col}"] = df.at[j, timing_col]
                        word_counters[col_word_key] = True
                        found = True
                        break
            if not found:
                all_found = False
                break
    
        if all_found:
            aligned_rows.append(match_info)
             # Mark the reference word as matched
            word_counters[word_key] = True
            
    # Create final DataFrame
    aligned_df = pd.DataFrame(aligned_rows)

    return aligned_df